In [2796]:
# imports
import pandas as pd
import numpy as np
import gradio as gr

In [2797]:
file_name = "data/googleplaystore.csv"
df_temp = pd.read_csv(file_name, encoding="utf-8")


In [2798]:
# เข้าใจข้อมูล
# check dataset info
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [2799]:
# check null
df_temp.isna().sum()
# จะเห็นได้ว่า rating มีข้อมูลเสียเยอะ

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [2800]:
# data prep & data cleaning
# เราจะใช้ข้อมูลที่จำเป็นสำหรับการใช้ข้อมูลนี้เท่านั้น
# ข้อมูลที่ไม่จำเป็นเราจะนำออกไป เช่น rating, reviews บลาๆ

df = df_temp.copy();

# remove unuse cols
df = df.reindex(columns=['App', 'Category', 'Size', 'Type', 'Genres', 'Price', 'Installs'])

In [2801]:
# rename col
temp_keys = {}
for i in df.columns:
    temp_keys[i] = i.lower()
df = df.rename(columns=temp_keys)

In [2802]:
df.isna().sum()
# ข้อมูลที่เสียมีน้อยมาก

app         0
category    0
size        0
type        1
genres      0
price       0
installs    0
dtype: int64

In [2803]:
# เราจะทำการลบข้อมูล ที่เสียหาย
df.dropna(axis=0 ,inplace=True)
df.isna().sum()

app         0
category    0
size        0
type        0
genres      0
price       0
installs    0
dtype: int64

In [2804]:
# แสดงข้อมูลเบื้องต้น
df.head(10)

,app,category,size,type,genres,price,installs
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,19M,Free,Art & Design,0,"10,000+"
1,Coloring book moana,ART_AND_DESIGN,14M,Free,Art & Design;Pretend Play,0,"500,000+"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,8.7M,Free,Art & Design,0,"5,000,000+"
3,Sketch - Draw & Paint,ART_AND_DESIGN,25M,Free,Art & Design,0,"50,000,000+"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,2.8M,Free,Art & Design;Creativity,0,"100,000+"
5,Paper flowers instructions,ART_AND_DESIGN,5.6M,Free,Art & Design,0,"50,000+"
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,19M,Free,Art & Design,0,"50,000+"
7,Infinite Painter,ART_AND_DESIGN,29M,Free,Art & Design,0,"1,000,000+"
8,Garden Coloring Book,ART_AND_DESIGN,33M,Free,Art & Design,0,"1,000,000+"
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,3.1M,Free,Art & Design;Creativity,0,"10,000+"


In [2805]:
# เช็คข้อมูล insight หลักๆ ที่อาจจะทำให้เรามีปัญหาในการใช้ข้อมูล
df.groupby(['category'])['category'].count()

category
1.9                       1
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     231
BUSINESS                460
COMICS                   60
COMMUNICATION           387
DATING                  234
EDUCATION               156
ENTERTAINMENT           149
EVENTS                   64
FAMILY                 1971
FINANCE                 366
FOOD_AND_DRINK          127
GAME                   1144
HEALTH_AND_FITNESS      341
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
LIFESTYLE               382
MAPS_AND_NAVIGATION     137
MEDICAL                 463
NEWS_AND_MAGAZINES      283
PARENTING                60
PERSONALIZATION         392
PHOTOGRAPHY             335
PRODUCTIVITY            424
SHOPPING                260
SOCIAL                  295
SPORTS                  384
TOOLS                   843
TRAVEL_AND_LOCAL        258
VIDEO_PLAYERS           175
WEATHER                  82
Name: category, dtype: int64

In [2806]:
# จะเห็นได้ว่า 1.9 นั้นไม่รู้คือ type อะไร
# และ genres, price,size installs นั้นสลับกันหมดเลย
# เรามาลองแก้กันดีกว่า
df.loc[df['category'] == '1.9']

,app,category,size,type,genres,price,installs
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,"1,000+",0,"February 11, 2018",Everyone,Free


In [2807]:
# ข้อมูลที่เราพบปัญหานั้น col ไม่ครบ เราเลยจะนำข้อมูลจากตารางหลักมา
# ข้อมูลจากตารางหลักบางอย่างนั้นไม่มี เช่น Category, Gengre 
# ซึ่งจากที่ดูๆ ข้อมูลเหมาะกับ HOUSE_AND_HOME, House & Home
# เราเลยจะเปลี่ยนข้อมูลให้กับข้อมูลนี้
temp_data = df_temp.loc[df_temp['App'] =='Life Made WI-Fi Touchscreen Photo Frame']
temp_data


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [2808]:
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'type'] = temp_data['Price']
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'size'] = temp_data['Category']
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'price'] = temp_data['Installs']
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'installs'] = temp_data['Size']
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'category'] = 'HOUSE_AND_HOME'
df.loc[df['app'] =='Life Made WI-Fi Touchscreen Photo Frame', 'genres'] = 'House & Home'

# เพื่อความแน่ใจ ลองเช็คข้อมูลอีกรอบ
df.loc[df['app'] == 'Life Made WI-Fi Touchscreen Photo Frame']

# ข้อมูลได้ถูกเปลี่ยนแล้วเย้


,app,category,size,type,genres,price,installs
10472,Life Made WI-Fi Touchscreen Photo Frame,HOUSE_AND_HOME,1.9,Everyone,House & Home,Free,"1,000+"


In [2809]:
# มาดูส่วนของ genres
df['genres'].head(5)

# จะเป็นได้ว่าบาง genres มีมา่กกว่า 1 รูปแบบ ซึ่งแบ่งด้วย ตัว semi colon

0                 Art & Design
1    Art & Design;Pretend Play
2                 Art & Design
3                 Art & Design
4      Art & Design;Creativity
Name: genres, dtype: object

In [2810]:
# เราจะแบ่งตัว genres ให้เป็นรูปแบบ ว่าตัว genres มันมีเท่าไหร่
genres = df['genres']
results = {}
for gen in genres:
    for g in gen.split(';'):
        if g not in results:
            results[g] = 1
            continue
        results[g] += 1
results


{'Art & Design': 69,
 'Pretend Play': 90,
 'Creativity': 37,
 'Action & Adventure': 138,
 'Auto & Vehicles': 85,
 'Beauty': 53,
 'Books & Reference': 234,
 'Business': 460,
 'Comics': 60,
 'Communication': 388,
 'Dating': 234,
 'Education': 761,
 'Music & Video': 47,
 'Brain Games': 70,
 'Entertainment': 667,
 'Events': 64,
 'Finance': 366,
 'Food & Drink': 127,
 'Health & Fitness': 343,
 'House & Home': 89,
 'Libraries & Demo': 85,
 'Lifestyle': 383,
 'Adventure': 91,
 'Arcade': 237,
 'Casual': 270,
 'Card': 51,
 'Action': 382,
 'Strategy': 110,
 'Puzzle': 167,
 'Sports': 402,
 'Music': 25,
 'Word': 29,
 'Racing': 119,
 'Simulation': 218,
 'Board': 63,
 'Trivia': 39,
 'Role Playing': 123,
 'Educational': 112,
 'Music & Audio': 1,
 'Video Players & Editors': 178,
 'Medical': 463,
 'Social': 295,
 'Shopping': 260,
 'Photography': 335,
 'Travel & Local': 258,
 'Tools': 843,
 'Personalization': 392,
 'Productivity': 424,
 'Parenting': 60,
 'Weather': 82,
 'News & Magazines': 283,
 'Maps &

In [2811]:
# จะเห็นได้ว่า installs เป็น type object ต้องเปลี่ยนเป็น int

df['installs'].sample(2)

9282       100+
6416    10,000+
Name: installs, dtype: object

In [2812]:
# ข้อมูลจะมี + ตามหลัง และ comma เราจะนำออก; และเปลี่ยนค่าเป็น int32
df['installs'] = df['installs'].str.replace('+', '', regex=True).replace(',', '', regex=True).astype(int)
df['installs']


0           10000
1          500000
2         5000000
3        50000000
4          100000
           ...   
10836        5000
10837         100
10838        1000
10839        1000
10840    10000000
Name: installs, Length: 10840, dtype: int32

In [2813]:
def convert_size(s):
    if s.replace('M', '').replace('.','',1).isdigit():
        size = float(s.replace('M', ''))
        return 1024 * size
    if s.replace('k', '').replace('.','',1).isdigit():
        return float(s.replace('k', ''))
    return 0


In [2814]:
# จะเห็นได้ว่า size เป็น type object ต้องเปลี่ยนเป็น int
# ซึ่งเราจะแปลกตัวเลขนี้ ให้เป็นค่า kb ทั้งหมด จะได้ใช้ได้ง่าย เมื่อมีค่าอิน นอกจากตัวค่า mb
# df['size'] = df['size'].str.replace('+', '', regex=True).replace(',', '', regex=True).astype(int)
df['size'] = df['size'].apply(convert_size)
df['size'].sample(5)


279     11264.0
3467        0.0
5293    44032.0
6078    12288.0
4595        0.0
Name: size, dtype: float64

In [2815]:
# price จะมีสองรูปแบบคือ free กับ paid ต้องเปลี่ยนเป็น bin ถึงจะได้ง่ายต่อการทำงาน

df.loc[df['price'] != "Free", 'price'] = df.loc[df['price']
                                                != "Free", 'price'].apply(lambda x: x.replace('$', ''))


df.loc[df['price'] == 'Free', 'price'] = 0

df['price'].sample(5)


1930       0
2625       0
8759       0
5945    2.99
1191       0
9846       0
8734       0
8840       0
7568       0
2043       0
Name: price, dtype: object